In [1]:
import pickle
import os
import csv
import numpy as np
# Specify the working directory
os.chdir('/Users/david/Desktop/FinetuneEmbed')
# os.chdir('/afs/crc.nd.edu/group/StatDataMine/dm011/Dailin_Gan/FinetuneEmbed')

from mod.mod_text import *

data_dir = "./data/long_vs_shortTF"
save_csv_dir = "./res/2024_1125/LongShortTF/long_vs_shortTF_finetune_auc.csv"
output_path = "./res/2024_1125/LongShortTF/LongShortTF_model_"

model_name = "sentence-transformers/all-MiniLM-L6-v2"
val_auc_ls = []
test_auc_ls = []

random_states = [1, 2]

/Users/david/anaconda3/envs/FinetuneBERT/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for random_state in random_states:
    output_dir = output_path  + str(random_state)

    train_dir = data_dir + "/TrainEvalTestData/train_data_" + str(random_state) + ".pkl"
    eva_dir = data_dir + "/TrainEvalTestData/eval_data_" + str(random_state) + ".pkl"
    test_dir = data_dir + "/TrainEvalTestData/test_data_" + str(random_state) + ".pkl"

    # prepare the input data
    with open(train_dir, "rb") as f:
        train_data = pickle.load(f)
    with open(eva_dir, "rb") as f:
        eval_data = pickle.load(f)
    with open(test_dir, "rb") as f:
        test_data = pickle.load(f)
        
    val_auc_scores = []
    
    train_texts, train_labels = train_data['desc'], train_data['labels']
    eval_texts, eval_labels = eval_data['desc'], eval_data['labels']
    test_texts, test_labels = test_data['desc'], test_data['labels']

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    mod_dir, val_auc = one_fold_training(train_texts, train_labels, eval_texts, eval_labels, 
                                     tokenizer, output_dir, 0, model_name)
    
    val_auc_scores.append(val_auc)

    # locate the best model
    best_fold_idx = np.argmax(val_auc_scores)
    best_model_dir = output_dir + '/fold_' + str(best_fold_idx + 1)
    best_model = AutoModelForSequenceClassification.from_pretrained(best_model_dir)

    # create the training data for final training
    train_texts_all = train_texts + eval_texts
    train_labels_all = train_labels + eval_labels

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    full_train_dataset = TextDataset(train_texts_all, train_labels_all, tokenizer)

    # Fine-tune the best model on the full training data
    output_dir = output_dir + "/final_model"
    final_trainer = finetune_best_mod(full_train_dataset, best_model, output_dir)

    # Evaluate the best model on the test set
    test_dataset = TextDataset(test_texts, test_labels, tokenizer)
    test_auc = pred_test(final_trainer, test_dataset)

    # save the results
    val_auc_ls.append(f"{val_auc:.4f}")
    test_auc_ls.append(f"{test_auc:.4f}")
    rows = zip(val_auc_ls, test_auc_ls)
    # Write to a CSV file
    with open(save_csv_dir, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Optional: Write a header row
        writer.writerow(['val_auc', 'test_auc'])
        # Write the rows
        writer.writerows(rows)


/Users/david/anaconda3/envs/FinetuneBERT/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
                                                
 20%|█▉        | 51/260 [00:13<02:02,  1.70it/s]

{'eval_loss': 0.6847690343856812, 'eval_AUC': 0.5427350427350427, 'eval_runtime': 1.4584, 'eval_samples_per_second': 23.998, 'eval_steps_per_second': 3.428, 'epoch': 3.85}


                                                 
 39%|███▉      | 101/260 [00:21<00:31,  5.06it/s]

{'eval_loss': 0.5700475573539734, 'eval_AUC': 0.561965811965812, 'eval_runtime': 0.1595, 'eval_samples_per_second': 219.436, 'eval_steps_per_second': 31.348, 'epoch': 7.69}


                                                 
 58%|█████▊    | 151/260 [00:28<00:20,  5.21it/s]

...

Fold 1 Validation AUC: 0.5918803418803419


100%|██████████| 360/360 [00:52<00:00,  6.80it/s]


{'train_runtime': 52.9358, 'train_samples_per_second': 52.516, 'train_steps_per_second': 6.801, 'train_loss': 0.8581428527832031, 'epoch': 20.0}


100%|██████████| 5/5 [00:00<00:00, 46.81it/s]

Test AUC: 0.4444444444444444
